In [1]:
import cv2 as cv2
import numpy as np
from wandb import Classes
import matplotlib.pyplot as plt

In [2]:
# load yolo
net = cv2.dnn.readNet("yolov3.weights","yolov3.cfg")
classes = []
with open("coco.names", 'r') as f:
    classes = [line.strip() for line in f.readlines()]
# print(classes)

cap = cv2.VideoCapture('Vehicle_count_test.mp4')


In [3]:
# Object tracking variables
vehicle_count = 0
previous_boxes = []

layer_name = net.getLayerNames()
output_layer = [layer_name[i - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [4]:
vehicle_count = 0
frame_interval = 2  # 2 seconds
frame_number = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    height, width, _ = frame.shape

    # Capture a frame every 2 seconds (30 frames per minute)
    if frame_number % (frame_interval * 30) == 0:
        # YOLO vehicle detection code here

        # Prepare input blob
        blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)

        net.setInput(blob)
        output_layers = net.getUnconnectedOutLayersNames()
        outs = net.forward(output_layers)

        class_ids = []
        confidences = []
        boxes = []

        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5 and class_id == 2:  # Class 2 represents "car" in COCO dataset
                    center_x, center_y, w, h = (np.array(detection[0:4]) * np.array([width, height, width, height])).astype(int)
                    x, y = int(center_x - w / 2), int(center_y - h / 2)
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                vehicle_count += 1

        print(f'Vehicles detected at frame {frame_number}: {vehicle_count}')

    frame_number += 1

    cv2.imshow('Vehicle Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

print(f'Total vehicles detected: {vehicle_count}')

Vehicles detected at frame 0: 3
Vehicles detected at frame 60: 10
Vehicles detected at frame 120: 19
Vehicles detected at frame 180: 28
Vehicles detected at frame 240: 33
Vehicles detected at frame 300: 36
Vehicles detected at frame 360: 42
Vehicles detected at frame 420: 47
Vehicles detected at frame 480: 48
Vehicles detected at frame 540: 54
Vehicles detected at frame 600: 57
Vehicles detected at frame 660: 62
Vehicles detected at frame 720: 64
Vehicles detected at frame 780: 70
Vehicles detected at frame 840: 73
Total vehicles detected: 73
